# Imports

In [113]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Functions

In [114]:
def add_sum_col(df):
    cols = df.columns
    for idx, col in enumerate(cols):
        df.loc['Summe',col] = df.iloc[:,idx].sum()

# Parameters

In [115]:
sbw_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\230928 Richterswil SK\Alle_exp_SBW\Exports\Teileinzugsgebiete_SBW.shp")
quartiere_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Bevölkerungsanalyse\Quartieranalyse\Daten\Quartieranalyse\QUARTIERE_F.shp")
sbw_tabelle = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\Richterswil_SBW-Tabelle.csv")

In [116]:
sbw = gpd.read_file(sbw_path)
quartiere = gpd.read_file(quartiere_path)
df_sbw = pd.read_csv(sbw_tabelle, sep=';')
sbw.rename(columns={'sbw_ref': 'SK ID'}, inplace=True)
sbw = pd.merge(sbw, df_sbw, on='SK ID')

# Einwohner Analysis

In [117]:
# EW dichte GEP
sbw['ew_dichte_gep'] = sbw['einwohner'] / sbw.area
print(sbw['einwohner'].sum())

# EW dichte Quartieranalyse
quartiere['ew_dichte_quartiere'] = (quartiere['U_EINW'] + quartiere['U_BESCH']) / quartiere.area
print(quartiere['U_EINW'].sum() + quartiere['U_BESCH'].sum())

12602.5021894399
17882


In [118]:
print(sbw.geometry.type.unique())
print(quartiere.geometry.type.unique())

sbw_quartiere = sbw.overlay(quartiere, how='intersection', keep_geom_type=None)
print(sbw_quartiere.geometry.type.unique())

['Polygon']
['Polygon']
['Polygon' 'MultiPolygon']


c:\Users\sru\repos\Private\hbt_projects\venv\lib\site-packages\geopandas\geodataframe.py:2469: UserWarning: `keep_geom_type=True` in overlay resulted in 258 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [119]:
sbw_quartiere['einwohner_gep'] = sbw_quartiere['ew_dichte_gep'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_gep'].sum())

sbw_quartiere['einwohner_quartiere'] = sbw_quartiere['ew_dichte_quartiere'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_quartiere'].sum())

11950.756506747257
16278.015599183695


## EW pro Sonderbauwerk

In [120]:
sbw_quartiere.set_index('Name', inplace=True)
ew_sbw = sbw_quartiere.groupby('Name')[['einwohner_gep', 'einwohner_quartiere']].sum()
add_sum_col(ew_sbw)
ew_sbw

,einwohner_gep,einwohner_quartiere
Name,,
ARA,594.219390,743.571286
PW Bruggeten,51.921284,68.526102
PW Klosterweg,155.077745,561.741020
PW Mülenen,159.875520,96.258712
PW Mülibach,609.355538,1111.061683
PW Schmitenbrunnen,67.264451,259.059746
RB Schulstrasse,901.077034,899.894027
RB Streuli,463.918815,556.399036
RB-PW Sagenbach,3.852021,2.462310


## EW pro Bauzone

In [121]:
name = 'U_ZONE_KT'
ew_bz = sbw_quartiere.groupby(name)[['einwohner_gep', 'einwohner_quartiere']].sum()
add_sum_col(ew_bz)
ew_bz

,einwohner_gep,einwohner_quartiere
U_ZONE_KT,,
IG-HD,402.418610,578.435656
IG-eHD,910.576008,1186.483168
K,1035.716954,2520.766874
Oe,1263.133047,385.529422
W2b,3668.939412,4878.133725
W3a,2705.578932,2985.655964
WG2b,86.690967,137.581274
WG3b,1877.702578,3605.429517
Summe,11950.756507,16278.015599


# Abflussbeiwert / Befestigungsgrad Analysis

In [122]:
file_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Grundlagen\EZG\ezg.xlsx")
ezg = pd.read_excel(file_path)

In [123]:
name = 'bauzone'
ezg.set_index(name, inplace=True)
ab_bg_ezg = ezg.groupby(name)[['abflussbeiwert_sw_ist', 'befestigungsgrad_sw_ist']].mean()
ab_bg_ezg

,abflussbeiwert_sw_ist,befestigungsgrad_sw_ist
bauzone,,
Bahnareal,0.000000,0.000000
Dreigeschossige Wohnzone,34.559756,23.193642
Dreigeschossige Wohnzone mit Gewerbeerleichterung,29.009485,18.646595
Erholungszone,13.272727,5.824800
Freihaltezone,24.777778,13.164621
Gewerbezone A,18.666667,13.212072
Gewerbezone B,35.000000,32.084699
Industriezone,36.452830,35.865663
Kantonale Freihaltezone,5.200000,2.062038
